### `singleton methods`

In [6]:
foo = Object.new

# can define a method directly on that object...
def foo.bar
  "bar"
end

:bar

### `defining methods for many instances of a class`

In [9]:
class Foo
  def bar
    "bar"
  end
end

:bar

Note a Class is an Object. And we know that objects are instances of a class. So Foo (a Class) is an instance of some class. And that class is Class. I know, it's weird.

In [14]:
Foo.is_a? Class

true

So in fact we can define a class dynamically using Ruby like this. It's actually equivalent to the code we had above.

In [15]:
Foo = Class.new do
  def bar
    "bar"
  end
end

(irb): warning: already initialized constant Object::Foo


#<Class:0x00007fe7809771f0>::Foo

In summary a Class is an object that has the power to spawn new objects. Nontheless they are objects themselves.

### `a Class is an object`

We interact with objects by sending messages. Since we know a Class is an object, we therefore send it messages.

### `how to send methods to a Class object`

Same syntax as how we've been sending messages to non-class objects (instances). Note: there are several ways to define a class method (define how a Class should respond to a specific message). You will see #3 being used a lot in the industry.

In [40]:
class Foo
  # 1
  def self.add_one(x)
    x + 1
  end
  
  # 2
  def Foo.add_two(x)
    x + 2
  end
  
  # 3
  class << self
    def add_three(x)
      x + 3
    end
  end
end

:add_three

In [30]:
p Foo.respond_to? :add_one
p Foo.send :add_one, 1

true
2


2

As an aside, how does #3 work. What is that weird syntax? Forget about the `self` keyword for one bit. And just look to whatever is on the right side of `<<`. We are essentially defining singleton methods on the object that is to the right of `<<`.

In [38]:
obj = Object.new
class << obj
  def yer
    'yerr!!!'
  end
end

:yer

In [39]:
obj.yer

"yerr!!!"

In [41]:
class Foo
  class << self
    def add_three(x)
      x + 3
    end
  end
end

:add_three

So if we have `class << self`, the `self` object in this context resolves into the enclosing class. Therefore, these two are equivalent.

```rb
class Foo
  class << self
    def add_three(x)
      x + 3
    end
  end
end

class Foo
  class << Foo
    def add_three(x)
      x + 3
    end
  end
end
```

### `are attr methods class methods?`

You betcha! The `attr_accessor` method is defined in a Class called Module.

In [61]:
class Foo
  '''
  note that there is a default receiver for each method call
  self.attr_accessor(:bar)
  ^ the self in this case is the enclosing class so Foo
  '''
  attr_accessor(:bar)
  
  p self.method(:attr_accessor)
  
  p Module.methods.include? :attr_accessor
  
end

#<Method: #<Class:#<Class:0x00007fe7809771f0>::Foo>(Module)#attr_accessor(*)>
true


true

In [60]:
Foo.class.superclass

Module

### `what the heck?`

Don't worry it's dead simple.

In [ ]:
### `classes as objects and message receivers`

Clases are special objects: they're the only kind of object that has the power to spawn new objects (instances). Nonetheless they're objects.

Like other objects, classes can be created. You can create a class object with the special `class` keyword formula.

To understand where classes get their methods, think about where objects in general get their methods.
  

- From their class
  
  
- From the superclass and earlier ancestors of their class
  
  
- From their own store of singleton methods `(def obj.talk)`
  
  
  
  # Class defines an instance method called `new`
# attr_accessor methods are defined in Module


# Class
#
# Person is an instance of Class, Class' superclass is Module...

Person = Class.new do
  def introduce
    p "Hello, I am a person"
  end
end

In [62]:
Foo.class.superclass

Module

### `what are class methods vs instance methods?`

In [52]:
class Foo
  class << self
    def try
    end    
  end
end

class Bar < Foo
  try
end

In [47]:
Foo.bar

NoMethodError: undefined method `bar' for #<Class:0x00007fe7809771f0>::Foo

In [48]:
Foo.class_methods

NoMethodError: undefined method `class_methods' for #<Class:0x00007fe7809771f0>::Foo
Did you mean?  alias_method

In [ ]:
### `module`

modules can either be included or extended w/ `include` or `extend`.

To include a module means to treat the methods that are included as instance methods.

To extend a module means to treat the extended methods as class methods.

In [ ]:
module Foo
  def foo
    puts "foo"
  end
end

class Baz
  include Foo
end

class Bar
  extend Foo
end

In [ ]:
### `modules are are adjectives`

In [ ]:
module Driveable # modules are also called mixins
end

module Stoppable
end

class Car # now a driveable car
  include Driveable
  include Stoppable
end

In [ ]:
car = Car.new

puts car.is_a? Driveable
puts car.is_a? Stoppable
puts car.is_a? Car
puts car.is_a? Object

In [ ]:
### `modules`

Unlike interfaces can contain state. Like abstract classes, can contains state.

Well they don't exactly contain state and neither do regular classes. Classes/modules contain instance methods - not state. State belongs to the object and get created when an instance method that sets state (setter) is run on the object.

Methods from a module that refer to an instance variable get mixed into class as instance methods. It's only when those instance methods are called on an objet that the instance variables are created - on that object. The instance variables don't belong to the module at all; they belong to the instances of the classes that mix the module.

In [ ]:
module AcceptsComments
  def comments
    @comments ||= []
  end
  
  def add_comment(comment)
    comments.push(comment)
  end
end

In [ ]:
class Video
  include AcceptsComments
end

In [ ]:
v = Video.new
v.add_comment("Yo")
v.add_comment("what")
v.comments

In [ ]:
### `modules should not have an initialize method`

Initialize method from the concrete class will override the initialize method from the mixin.
    
When you mix a module into the class, Ruby will look for methods (both mixins and superclass) by using the ancestors class method. It will return an array with all of the class' mixins and superclasses in the order they will be searched.

In [ ]:
class MySuperClass
  def foo
  end
end

module MyModule
  def foo
  end
end

class MyClass < MySuperClass
  include MyModule
  def foo 
  end
end

In [ ]:
p MyClass.ancestors

In [ ]:
class MySuperClass
  def foo
    puts "MySuperClass"    
  end
end

module MyModule
  def foo
    puts "MyModule"
  end
end

class MyClass < MySuperClass
  include MyModule
  def foo
    super
    puts "MyClass"    
  end
end

In [ ]:
MyClass.new.foo

In [ ]:
### `top level`

The default receiver is `self`. As you can see there are many methods which are available to self in the top-level execution context.

We can mixin methods at different levels, including the top-level.

In [ ]:
module Bespoke
  def suit_up
    p "Yes, sir"
  end
end

include Bespoke
suit_up

self.methods.include? :suit_up

p self

In [ ]:
### `every method call has a receiver`

In [ ]:
# even this has a receiver (implicitly a self receiver)
puts "Hello"

self.puts "Hello"

### `respond_to? / send`

In [ ]:
foo = Object.new

def foo.bar
    "bar"
end

In [ ]:
p foo.respond_to? :bar
p foo.respond_to? "bar"

p foo.send("bar")
p foo.send(:bar)

### `defined?`

In [3]:
class Skirt
end

defined? Skirt

"constant"

In [4]:
module Yer
end

defined? Yer

"constant"

In [1]:
def baz
end

defined? baz

"method"

In [2]:
abra = 1

defined? abra

"local-variable"

In [5]:
defined? self

"self"